## Creating a SoftMax Neural Net from Scratch

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plot 
import seaborn 
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

In [3]:
#Load the data
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train, x_test, y_test = x_train.astype('int'), y_train.astype('int'), x_test.astype('int'), y_test.astype('int')

In [2]:
#Data preparation section

def create_one_hot_train_and_test(y_train, y_test):
    one_hot = OneHotEncoder(sparse = False)
    one_hot_y_train= one_hot.fit_transform(y_train.reshape(-1,1)).T ###currently doing it on two classes
    one_hot_y_test = one_hot.fit_transform(y_test.reshape(-1,1)).T
    return one_hot_y_train, one_hot_y_test

def flatten_and_normalise_x_values(x_train, x_test):
    x_train_flat = x_train.reshape((x_train.shape[0],-1)).T / 255
    x_test_flat = x_test.reshape((x_test.shape[0],-1)).T / 255
    return x_train_flat, x_test_flat

In [5]:
x_train_flat, x_test_flat = flatten_and_normalise_x_values(x_train,x_test)
one_hot_y_train, one_hot_y_test = create_one_hot_train_and_test(y_train, y_test)

In [119]:
class Neural_Net_Softmax():
    
    #Initialise with the input shape and output shape
    def __init__(self, input_features, output_features):
        self.input_dimension = input_features
        self.output_dimensions = output_features
        #Initialise the dictionaries here so we can't accidentally store values we don't need if we change the input sizes.
        self.fp_dict = {}
        self.bp_dict = {}

    def create_layers(self,array_of_mid_layer_sizes, activation_functions = 'relu'):
        self.mid_layer_sizes = array_of_mid_layer_sizes        
        
        #for layer type, we can either set all layers to one type or have an array of layers passed
        if activation_functions in ('relu', 'sigmoid'):
            self.mid_layer_types = [activation_functions] * len(array_of_mid_layer_sizes)
        else:
            self.mid_layer_types = activation_functions
        self.fp_rounds = len(array_of_mid_layer_sizes) + 1 #Define this here as will be used a lot later and creates one terminology
        self.all_layers = [self.input_dimension] + self.mid_layer_sizes + [self.output_dimensions] #

    def init_weights(self, init_methods='gaussian'):
        self.weights = {}
        
        #Allow either a list of init_methods or one init method for all layers
        if type(init_methods) == str:
            init_methods = [init_methods] * self.fp_rounds #This creates a list of layers-1 weight initialisation strings.
        
        #this iterates through the size of layer and the init methods
        for layer, init_methods in zip(range(self.fp_rounds),init_methods):
            output_dim = self.all_layers[layer+1]
            input_dim = self.all_layers[layer] 

            #Use locals to call the reelvant function
            self.weights[f'W{layer+1}'], self.weights[f'B{layer+1}'] = globals()[f'_{init_methods}_init'](output_dim, input_dim)
        return self.weights
    
    def forward_prop(self, x_flat, one_hot_y, regularisation = False, gradient_smoother = False):
        self.fp_dict['A0'], self.fp_dict['Z0'] = x_flat,0 #helpful for iteratting through a's and Z's in backprop
        fp_rounds = self.fp_rounds

        #First, do all the layers up to thte softmax layer ,which has a different activation function.
        for key, activation_function in zip(range(1,fp_rounds),self.mid_layer_types): #starts at 1, goes up to the last hidden layer
            self.fp_dict[f'Z{key}'], self.fp_dict[f'A{key}'] = _sigmoid_fp(self.fp_dict[f'A{key-1}'], self.weights[f'W{key}'], self.weights[f'B{key}'])

        #Then do the final layer and calculate cost
        self.fp_dict[f'Z{fp_rounds}'], self.fp_dict[f'A{fp_rounds}'] = _last_leg_softmax(self.fp_dict[f'A{fp_rounds-1}'], self.weights[f'W{fp_rounds}'], self.weights[f'B{fp_rounds}']) 
        cost = -np.sum(one_hot_y * np.log(self.fp_dict[f'A{fp_rounds}'])) / one_hot_y.shape[1] #divides by training examples
        return self.fp_dict, cost

    def back_prop(self, x_flat, one_hot_y):
        ZA_dict = {} #this is a temporary dict to store values that are needed at later stages of back prop.
        ZA_dict['Z0'] = {} #Just useful for iterations.
        
        #The first dC_dZ is the classic Softmax function, ÿ - y
        ZA_dict[f'dC_dZ{self.fp_rounds}'] = self.fp_dict[f'A{self.fp_rounds}'] - one_hot_y
        
        #Then we need to iterate back across all layers, getting the relevant dw's for grad descent and the dZs for the previous layer        
        final_layer_code = 'no' #this is to solve the issue of the last layer not having Z values
        print(self.fp_rounds+1)
        for key in reversed(range(1,self.fp_rounds+1)): #e.g. for a network with 5 layers (and 5 Ws, 5 Bs), this will iterate 5,4,3,2,1 and on 1 it will have final layer code of 'yes'
            if key == 1:
                final_layer_code = 'yes'
            self.bp_dict[f'dW{key}'], self.bp_dict[f'dB{key}'], ZA_dict[f'dC_dZ{key-1}'] = back_prop_from_z_to_z_sigmoid(ZA_dict[f'dC_dZ{key}'], final_layer_code, self.fp_dict[f'Z{key-1}'], self.fp_dict[f'A{key-1}'], self.weights[f'W{key}'])
        return self.bp_dict

    def gradient_descent(self, learning_rate):


'''Helper functions - outside of class so they can be called with globals'''

#weights initialisati"ons
def _gaussian_init(output_dim, input_dim):
    W, B = np.random.randn(output_dim,input_dim), np.random.randn(output_dim,1)
    return W,B
def _xavier_init(output_dim,input_dim):
    W, B = np.random.uniform(-np.sqrt(6)/(np.sqrt(output_dim+input_dim)),np.sqrt(6)/(np.sqrt(output_dim+input_dim)),[output_dim,input_dim]), np.random.uniform(-np.sqrt(6)/(np.sqrt(output_dim+1)),np.sqrt(6)/(np.sqrt(output_dim+1)),[output_dim,1])
    return W,B
#add relu     

#forward props
def _sigmoid(x):
    return (1/(1+np.exp(-x)))
def _sigmoid_fp(prev_a, W, B):
    Z = W @ prev_a + B
    return Z, _sigmoid(Z)    
def _relu(x):
    return(max(0,x))
def _relu_fp(prev_a, W, B):
    Z = W @ prev_a + B
    return Z, _relu(Z)
def _last_leg_softmax(previous_a, W, B):
    Z = W @ previous_a + B
    A = np.exp(Z)
    A = A / np.sum(A,axis=0) 
    return Z, A   

## BACK PROPOGATION 
def back_prop_from_z_to_z_sigmoid(dC_dZn, final_layer_or_not,prev_Z, prev_A,W):    

    #get diffs at that layer
    dZn_dWn = prev_A
    dZn_dBn = np.zeros_like(dC_dZn) + 1 #output feature x training examples
    #combien to get useful differentials
    dC_dWn = dC_dZn @ dZn_dWn.T / dC_dZn.shape[1] #training examples
    dC_dBn = np.sum(dC_dZn @ dZn_dBn.T,axis=1,keepdims=True) / dZn_dBn.size ###THIS COULD BE WRONG, MIGHT BE SHAPE. let's try both. BUT IT LOOKS like this is creating a column created from mxmx10, which is shape.

    #and then get the previous layer cost fuctions
    if final_layer_or_not == 'no':
        dZn_dA_minus_one = W
        dC__dA_minus_one = (dC_dZn.T @ dZn_dA_minus_one).T
        dAn_minus_one__dZ_minus_one = np.exp(-prev_Z) / ((1+np.exp(-prev_Z))**2) #sigmoid differentiated
        dC__dZ_minus_one = dC__dA_minus_one * dAn_minus_one__dZ_minus_one
        return dC_dWn, dC_dBn, dC__dZ_minus_one
    else:
        return dC_dWn, dC_dBn, 0

Unit test 1 - check whether xavier / He gives the right mean and variance of Z1

In [121]:
nn_test = Neural_Net_Softmax(784, 10)
nn_test.create_layers([10,10], 'relu')
nn_test.init_weights()
nn_test.forward_prop(x_train_flat,one_hot_y_train)
nn_test.back_prop(x_train_flat, one_hot_y_train)
(nn_test.all_layers)

4
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [[ 0.00020314]
 [-0.01451835]
 [-0.00124917]
 [-0.00244308]
 [ 0.00676162]
 [ 0.00449899]
 [-0.00086447]
 [ 0.0012348 ]
 [ 0.00323632]
 [-0.01618736]] 0 [[ 5.06864432e-02  4.58890848e-04  1.00387101e-07 ...  1.84230343e-03
  -6.81064196e-03  1.11950022e-04]
 [-2.99336097e-02  3.59145214e-04  3.23328599e-02 ... -2.51820238e-01
  -2.40344363e-03  3.71287082e-07]
 [ 1.87179945e-05 -6.76004398e-04  9.52350718e-02 ...  1.48308409e-01
   3.77855717e-03 -1.53329816e-01]
 ...
 [ 1.04893545e-03 -9.69362619e-03  3.74122085e-02 ...  1.86835583e-03
  -5.20870357e-09 -1.63459276e-06]
 [ 3.50679475e-02 -9.75270098e-02 -1.57924894e-02 ...  1.07850918e-04
  -8.82440465e-03  1.87803646e-03]
 [-9.26014119e-04  1.15360171e-04 -3.24386600e-03 ...  1.41772972e-02
   4.52061169e-06 -5.60522790e-04]] yes 0 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0

[784, 10, 10, 10]

In [138]:
pretend_relu = [
[-1,1,-1],
[2,2,3],
[-10,2,100]
]

def relu_function(Z):
    return(np.maximum(Z,np.zeros_like(Z)))
relu_function(pretend_relu)


array([[  0,   1,   0],
       [  2,   2,   3],
       [  0,   2, 100]])